# Lecture 8 - Time Series Analysis

In this notebook we will learn how to perform time series analysis of Twitter data. 

<ol type = 1>
<li> Data processing</li>
    <ol type = a>
    <li> Load users' tweets</li>
    <li> Convert tweet times into datetime objects</li>
    <li> Shift times of tweets</li>
    </ol>
<li> Tweet rate</li>
    <ol type = a>
    <li> Calculate mean tweet rate</li>
    <li> Calculate rolling average tweet rate</li>
        <li> Word clouds of tweets in a specific window </li>
<li>Calculate rolling average of retweet count </li>
    </ol>
 
<li>Tweet hours and days</li>

<ol type = a>
<li>Histograms of tweet hour and day of the week </li>
<li> Word clouds for tweets on a certain hour or day of the week </li>
</ol>

<li> Estimate a user's time zone using high low model </li>
</ol>
</ol>

This notebook can be opened in Colab 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zlisto/social_media_analytics/blob/main/Lecture08_TimeSeriesAnalysis.ipynb)

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.

# Clones, installs, imports, and GPU


## Clone GitHub Repository
This will clone the repository to your machine.  This includes the code and data files.  Then change into the directory of the repository.

In [1]:
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")

Cloning into 'social_media_analytics'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 172 (delta 93), reused 114 (delta 38), pack-reused 0
Receiving objects: 100% (172/172), 11.63 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (93/93), done.


## Install Requirements 

In [ ]:
!pip install -r requirements.txt



     |████████████████████████████████| 58 kB 3.9 MB/s 
     |████████████████████████████████| 3.5 MB 19.9 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
     |████████████████████████████████| 1.6 MB 28.7 MB/s 
     |████████████████████████████████| 6.8 MB 37.4 MB/s 


## Import Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import scripts.TextAnalysis as ta
from scripts.api import *

# Data Processing

##  Load data

Load the tweets in the file `"data/tweets_lec_10.csv""` into a dataframe. There are many users in this dataframe


In [ ]:
fname_db = "data/lecture_04"
df_all = DB.fetch(table_name = 'user_tweets', path = fname_db)
df_all.screen_name.unique()

## Select Tweets of  a Screen Name and Keep Relevant Columns

Choose a `screen_name` from those in `df_all`.  We will study this user's tweets.

There are many columns in this dataframe, but we only need a few today.  We will keep only the columns `screen_name`, `created_at`, `text`, and `retweet_count`.  

We will also be making word clouds, so apply the `clean_tweet` function to the `text` column and add this as a new column `text_clean` to the dataframe.


In [ ]:
screen_name = 'kanyewest'

df = df_all[df_all.screen_name==screen_name]
df = df[['screen_name','created_at','text','retweet_count']]
df['text_clean'] = df.text.apply(ta.clean_tweet)  #clean the tweets

ntweets = len(df)
print(f"dataframe has {ntweets} tweets")
df.head()

## Datetimes

Right now the tweet time is in the `created_at` column as a string.  To do time series analysis we have to convert it to a `datetime` object.  This can be done using the `to_datetime` function.  We have to specify the format of the date and time in the string.  We do this using the string variable `format`.  The `created_at` format is "year-month-date hour:minute:second".  In Python, this format is defined as `format = "%Y-%m-%d %H:%M:%S"`. 

We add a column `created_at_datetime` to the dataframe which are the `created_at` column converted to datetimes.  

In [ ]:
format = "%Y-%m-%d %H:%M:%S"
df['created_at_datetime'] = pd.to_datetime(df['created_at'],format=format).dt.tz_localize(None)
df.head()

## Shifting Timezone

The `created_at` field is in Coordinated Universal Time (UTC), which is the time in London.  If you want to shift the time to a different timezon, just add your local offset.

For example, New York is -4 UTC, so we just add -4 hours to the UTC time.  This can be done using `timedelata64` object.  You specify you want hours with the string `'h'`, and the number of hours.  We will add this column to the dataframe and call it `created_at_datetime_ny`.

In [ ]:
df['created_at_datetime_ny'] = df['created_at_datetime'] + np.timedelta64(-4, 'h')
df.head()

## Heads and Tails

It's nice to sort the dataframe rows by the `created_at_datetime` column.  We can do this using the `sort_values` function.  Sort with `ascending=True` so that the first rows are the earliest tweets. Use the option `inplace=True` to have the dataframe be equal to its sorted version. 

To look at the first rows in the dataframe we use the `head` command.

To look at the last rows in the dataframe we use the `tail` command.

In [ ]:
df.sort_values(by = 'created_at_datetime', inplace = True, ascending = True)
print(f"Head:\n{df.created_at.head()}")
print(f"Tail:\n{df.created_at.tail()}")



# Tweet Rate

## Mean Tweet Rate

To calculate the mean tweet rate, we find the number of tweets and divide it by the duration they span.  The number of tweets `ntweets` is just the number of rows in the dataframe, which you can obtain using the `len` function.  The duration of the tweets `duration` is found by subtracting the datetime of the earliest tweet from the datetime of the most recent tweets.  

To obtain the earliest time we can use the `min` function.  To obtain the most recent time we can use the `max` function.

`duration` is a `TimeDelta` object.  To make things simple, we convert it to a number of days using the `days` parameter.

The `rate_estimate` is `ntweets` divided by `duration`.



In [ ]:
ntweets = len(df)  #number of tweets

duration = df.created_at_datetime.max()-df.created_at_datetime.min()  #duration in TimeDelta
duration = duration.days #duration in days

rate_estimate = ntweets/duration  #rate [tweets/day]

print('Rate estimate = %.2f tweets per day'%(rate_estimate))


## Rolling Window Tweet Rate

To obtain the rolling window tweet rate, we can make use of the `rolling` function.  This function creates a dataframe with rows grouped together if they are in a window specified by the row.  The `rolling` function is sort of like the `groupby` function we used before.

We will specify the windows as being a fixed amount of time before the `created_at_datetime` column.  If we wanted a 1 day window, we use the format `rolling('1D',on='created_at_datetime'`.  If we wanted a 7 day window, we use the format `rolling('7D',on='created_at_datetime'`. After applying `rolling`, we need to apply another function to get the rate.  We want to know the number of tweets in the window.  To do this we create a column `tweet_indicator` that is 1 for every tweet.  Then to get the number of tweets in the window we apply`sum` after `rolling` to the `tweet_indicator` column.  

The rate will be in units of tweets/day. If the window is 1 day, then after applying `sum` you have the rate in the proper units.  If the window is 7 days, you need to divide the `sum` by 7 to get the proper units.

We add the rates to the dataframe as columns called `rate_1D` and `rate_7D`.



In [ ]:
df['tweet_indicator'] = np.ones(len(df))
df['rate_1D'] = (df.rolling('1D',on = 'created_at_datetime').sum()['tweet_indicator']-1)*1
df['rate_7D'] = (df.rolling('7D',on = 'created_at_datetime').sum()['tweet_indicator']-1)*1/7



## Plot Rate

The `lineplot` function allows us to plot the rate with different window sizes.  You can set the x-axis limits with the `xlim` command. The values are datetime objects, which are specified by `datetime.date(year,month,date)`

In [ ]:
fig = plt.figure(figsize = (12,6))
sns.lineplot(data = df, x = 'created_at_datetime',y = 'rate_1D',label = 'window = 1 day',linewidth = .25,color = 'blue')
sns.lineplot(data = df, x = 'created_at_datetime',y = 'rate_7D',label = 'window = 7 days', linewidth = 2,color = 'purple')

plt.plot(df.created_at_datetime.tolist(),rate_estimate*np.ones(len(df)),label = 'mean rate',linestyle = '--',linewidth = 2,color = 'black')

plt.grid()
plt.ylabel("Rate [tweets/day]",fontsize = 16)
#plt.xlim(datetime.date(2021, 1, 1),datetime.date(2022,1,1))
#plt.ylim([0,30])
plt.title(f"{screen_name}",fontsize = 20)

plt.legend()
plt.show()

## Tweets During Peak Tweet Rate

Let's find the datetime when the tweet rate with a 1 day window is maximal and call this value `tpeak`.  We do this with the logical condition `df.rate_1D==df.rate_1D.max()`, and then getting the value in the `created_at_datetime` column.

In [ ]:
tpeak = df[df.rate_1D==df.rate_1D.max()].created_at_datetime.values[0]
print(f"Peak tweet rate on {tpeak}")


## Peak Rate Tweets

We can make a dataframe of the tweets within a 24 hour window ending at `tpeak`.  We specify `t0` and `t1` as the start and stop stime for the window.  Then we use the logical condition  `(df.created_at_datetime>=t0) & (df.created_at_datetime<t1)` to get all tweets in this window and call the resulting dataframe `df1`.



In [ ]:
t0 = tpeak - np.timedelta64(24, 'h')
t1 = tpeak 
df1 = df[(df.created_at_datetime>=t0) & (df.created_at_datetime<t1)]

df1.head()


## Word Cloud of Peak Tweets

We can make a wordcloud of the peak rate tweets to understand what was driving the increased activity that day.

In [ ]:
stopwords = set(STOPWORDS)
text=' '.join(df1.text_clean.tolist()).lower()
wordcloud = WordCloud(stopwords=stopwords,max_font_size=150, max_words=100, background_color="white",width=1000, height=600)
wordcloud.generate(text)

fig = plt.figure(figsize = (6,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Rolling Average of Retweet Count

We can use the `rolling` function to plot rolling averages of numerical quantities like `retweet_count`.  We just apply the `mean` function.  

In [ ]:
df['retweet_count_1D'] = df.rolling('1D',on = 'created_at_datetime').mean()['retweet_count']
df['retweet_count_7D'] = df.rolling('7D',on = 'created_at_datetime').mean()['retweet_count']


In [ ]:
fig = plt.figure(figsize = (12,6))
sns.lineplot(data = df, x = 'created_at_datetime',y = 'retweet_count_1D',label = 'window = 1 day',
             linewidth = .25,color = 'blue')
sns.lineplot(data = df, x = 'created_at_datetime',y = 'retweet_count_7D',label = 'window = 7 day',
             linewidth = 1,color = 'purple')


plt.grid()
plt.ylabel("Retweet count",fontsize = 16)
#plt.xlim(datetime.date(2014, 1, 1),datetime.date(2021,1,1))
#plt.ylim([0,1e5])
plt.legend()
plt.title(f"{screen_name}",fontsize = 20)
plt.show()

# Tweet Hours and Days

##  Histogram of Tweet Day of the Week
Create a column in the dataframe called `day` which has the day of the week of each tweet.  We get the day of the week of a tweet with the `dt.day_name()` function.  

We can then make a histogram of the day of the user's tweets. 

In [ ]:
df['day'] = df.created_at_datetime.dt.day_name()   #get the day of the tweet

fig = plt.figure(figsize = (10,4))
sns.histplot(data = df, x = 'day')
plt.title(f"{screen_name}",fontsize = 20)
plt.ylabel("Tweet count",fontsize = 14)
plt.xlabel("Day",fontsize = 14)
plt.grid()
plt.show()

##  Tweet Hour Histograms

Create a column `hour` with the hour of the tweets.  We can obtain the hour of a tweet with the `hour` parameter on the `datetime_created_at` column.  Note that the hours will be in UTC.

Then we can make a histogram of the hour of each tweet for each user in the dataframe.  We will force the histogram to have have 24 bins going from 0 to 23 with the `bins` variable and the `bins` parameter in this `histplot` function.


In [ ]:
df['hour'] = df.created_at_datetime.dt.hour   #get the day of the tweet

bins = list(range(0,24))


fig = plt.figure(figsize = (10,4))
sns.histplot(data = df, x = 'hour',bins = bins)
plt.title(f"{screen_name}",fontsize = 20)
plt.ylabel("Tweet count",fontsize = 14)
plt.xlabel("Hour [UTC]",fontsize = 14)
plt.grid()
plt.show()

## Tweets During a Specific Hour or Day

We can get the tweets posted during a specific hour (i.e. 10 AM) or day of the week (i.e. Monday) using simple logical conditions applied to the `day` and `hour` columns.

In [ ]:
df_hour = df[df.hour==10]
df_hour.head()

In [ ]:
day = 'Wednesday'
df_day = df[df.day==day]
df_day.head()

## Word Cloud of Tweets on a Specific Hour or Day

Once you create a dataframe with tweets from a specific time period, hour, or day, you can easily make a word cloud of them.

In [ ]:
stopwords = set(STOPWORDS)
stopwords.add(screen_name)
text=' '.join(df_day.text_clean.tolist()).lower()
wordcloud = WordCloud(stopwords=stopwords,max_font_size=150, max_words=100, background_color="white",width=1000, height=600)
wordcloud.generate(text)

fig = plt.figure(figsize = (6,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title(f"Tweets of {screen_name} on {day}",fontsize = 20)

plt.show()

# Geo-location with Tweet Times 

Now we will try to figure out what time zone people are based on when their Twitter activity is low.

##  Tweet Hour Histograms

Let's begin by making a histogram of the hour of each tweet for each user in the dataframe. We will specify the number of bins, as we did before.



In [ ]:
bins = list(range(0,24))

for screen_name in df.screen_name.unique():
    df1 = df[df.screen_name==screen_name]
    fig = plt.figure(figsize = (10,4))
    sns.histplot(data = df1, x = 'hour',bins = bins)
    plt.title(f"{screen_name}",fontsize = 20)

    plt.ylabel("Tweet count",fontsize = 14)
    plt.xlabel("Hour",fontsize = 14)
    plt.grid()
    plt.show()  

## Function to Find UTC Offset based on High Low Model

The function `geolocate_utc_offset` takes as input a dataframe of tweets `df` and a list of hours when activity is low, `low_hours` and returns the UTC offset that puts the fewest number of tweets in the low activity period.  You need a column `hour` in the dataframe that has the hour of each tweet.  

`geolocate_utc_offset` returns three things:

1. `utc_offset_min` = UTC offset to put fewest tweets in `low_hours`

2. `nlows` = list of number of tweets during `low_hours` for each possible UTC offset

3. `utc_offsets` = list of possible UTC offsets

In [ ]:
def geolocate_utc_offset(df,low_hours):
    nlows = []
    utc_offsets = list(range(-11,13))
    for utc_offset in utc_offsets:
        low_hours_offset = [(h-utc_offset)%24 for h in low_hours]
        nlow = len(df1[df1.hour.isin(low_hours_offset)])
        nlows.append(nlow)
    utc_offset_min = utc_offsets[nlows.index(min(nlows))]
    return utc_offset_min,nlows,utc_offsets

## Find UTC Offset

We specify `low_hours` as a list of hours when we think activity is low.  Then a `for` loop goes through each screen name and finds their UTC offset.  This `for` loop might come in handy on your homework :)

To see possible cities in the time zone corresponding to the UTC offset, use this link:

https://www.timeanddate.com/time/map/

In [ ]:
low_hours = [4,5]  #hours when activity is low in local time zone 
for screen_name in df.screen_name.unique():
    df1 = df[df.screen_name==screen_name]
    utc_offset_min,nlows,utc_offsets = geolocate_utc_offset(df1,low_hours)
    print(f"{screen_name}: UTC offset = {utc_offset_min}")